# CLSmith Results

To generate results:

```
# make programs
$ ./mkprograms -H cc1 -n 10000
# list available OpenCL platforms and devices
$ ./clinfo
# run programs on platform ID and device ID
$ ./runprograms -H cc1 0 0
```

Connect to results database:

In [1]:
import db
from db import Program, Testbed, Params, Result, Session

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/clsmith'

## Overview

In [2]:
import pandas as pd

data = []
with Session() as session:
    data.append(("Programs", session.query(Program).count()))
    data.append(("Testbeds", session.query(Testbed).count()))
    data.append(("Params", session.query(Params).count()))
    data.append(("Results", session.query(Result).count()))
pd.DataFrame(data, columns=["", "Count"])

,,Count
0,Programs,10001
1,Testbeds,2
2,Params,3
3,Results,21170


### TestBeds

In [3]:
testbeds = []
with Session() as session:
    for testbed in session.query(Testbed):
        nresult_tb = session.query(Result).filter(Result.testbed_id == testbed.id).count()
        ratio_result = "{:.2%}".format(nresult_tb / session.query(Result).count())
        testbeds.append((testbed.id, testbed.device, testbed.driver, nresult_tb, ratio_result))

pd.DataFrame(testbeds, columns=["ID", "Name", "Driver", "#. of Results", "% of Total Results"])

,ID,Name,Driver,#. of Results,% of Total Results
0,9,Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz,1.2.0.25,12347,58.32%
1,3,GeForce GTX 1080,375.39,8823,41.68%


### Parameters

In [4]:
params = []
with Session() as session:
    for param in session.query(Params).order_by(
            Params.gsize_x, Params.gsize_y, Params.gsize_z,
            Params.lsize_x, Params.lsize_y, Params.lsize_z, Params.optimizations):
        nresult_param = session.query(Result).filter(Result.params_id == param.id).count()
        ratio_result = "{:.2%}".format(nresult_param / session.query(Result).count())
        print(param)
        params.append((param.id, param.gsize, param.lsize, param.optimizations_on_off, nresult_param, ratio_result))

pd.DataFrame(params, columns=["ID", "Global size", "Local size", "Optimizations", "#. of Results", "% of Total Results"])

Optimizations: on, Global size: (1, 1, 1), Local size: (1, 1, 1)
Optimizations: off, Global size: (128, 16, 1), Local size: (32, 1, 1)
Optimizations: on, Global size: (128, 16, 1), Local size: (32, 1, 1)


,ID,Global size,Local size,Optimizations,#. of Results,% of Total Results
0,3,"(1, 1, 1)","(1, 1, 1)",on,39,0.18%
1,2,"(128, 16, 1)","(32, 1, 1)",off,2372,11.20%
2,1,"(128, 16, 1)","(32, 1, 1)",on,18759,88.61%


### Result Exit Status

In [5]:
import sqlalchemy as sql

# exit code descriptions
DESCRIPTIONS = {
    -9: "Timeout",
    0: "Okay"
}

statuses = []
with db.Session() as session:
    nresult = session.query(Result).count()
    q = session.query(
        Result.status, sql.func.count(Result.status)).group_by(Result.status).order_by(Result.status)
    
    for status, count in q.all():
        ratio = "{:.2%}".format(count / nresult)
        statuses.append((status, DESCRIPTIONS.get(status, ""), count, ratio))

pd.DataFrame(statuses, columns=["Status", "Description", "#. of Results", "% of Total Results"])      

,Status,Description,#. of Results,% of Total Results
0,-11,,239,1.13%
1,-9,Timeout,1656,7.82%
2,0,Okay,15714,74.23%
3,1,,3561,16.82%


In [6]:
with db.Session() as session:
    for param in session.query(db.Params).all():
        suspicious = []

        print("Params:", param)
        programs_with_results = session.query(db.Program).filter(
            db.Program.id.in_(session.query(db.Result.program_id).filter(db.Result.params_id == param.id)))
        programs_with_good_results = session.query(db.Program).filter(
            db.Program.id.in_(session.query(db.Result.program_id).filter(
                db.Result.params_id == param.id, db.Result.status == 0)))
        ratio_good = programs_with_good_results.count() / programs_with_results.count()

        print("  #. programs with results", programs_with_results.count())
        print("  #. programs with good results", programs_with_good_results.count(),
              "({ratio_good:.1%} of total)".format(**vars()))
        
        for program in programs_with_good_results.all():
            q = session.query(db.Result).filter(
                    db.Result.program_id == program.id, db.Result.params_id == param.id)
            
            stdout = set(r.stdout for r in q.all())
            if len(stdout) > 1:
                suspicious.append(q.first().program_id)
        
        nsuspicious = len(suspicious)
        ratio_suspicious = nsuspicious / programs_with_good_results.count()
        print("  #. suspicious {nsuspicious} ({ratio_suspicious:.1%} of total)\n".format(**vars()))

Params: Optimizations: off, Global size: (128, 16, 1), Local size: (32, 1, 1)
  #. programs with results 2354
  #. programs with good results 1585 (67.3% of total)
  #. suspicious 1 (0.1% of total)

Params: Optimizations: on, Global size: (1, 1, 1), Local size: (1, 1, 1)
  #. programs with results 39
  #. programs with good results 34 (87.2% of total)
  #. suspicious 0 (0.0% of total)

Params: Optimizations: on, Global size: (128, 16, 1), Local size: (32, 1, 1)
  #. programs with results 10001
  #. programs with good results 8109 (81.1% of total)
  #. suspicious 1301 (16.0% of total)

